In [47]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [48]:
market_value_df = pd.read_csv('Data/Monthly - Market Value.csv')
market_value_df['date'] = pd.to_datetime(market_value_df['date']).dt.to_period('M').astype(str)

In [49]:
momentum_df = pd.read_csv('Data/Monthly - Momentum.csv')
momentum_df['date'] = pd.to_datetime(momentum_df['date']).dt.to_period('M').astype(str)

In [50]:
merged_df = pd.merge(market_value_df, momentum_df, on=['date', 'TICKER', 'PERMNO', 'SICCD', 'COMNAM', 'NAICS', 'PRC'], how='inner')

In [5]:
merged_df.to_csv('Data/Monthly_Merged.csv', index=False)

In [51]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   PERMNO  117558 non-null  int64  
 1   date    117558 non-null  object 
 2   SICCD   117442 non-null  float64
 3   TICKER  117144 non-null  object 
 4   COMNAM  117442 non-null  object 
 5   NAICS   114872 non-null  float64
 6   PRC     117064 non-null  float64
 7   SHROUT  117428 non-null  float64
 8   VOL     117135 non-null  float64
 9   RET     117362 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 9.0+ MB
None


In [52]:
missing_values = merged_df.isnull().sum()
print(missing_values)

PERMNO       0
date         0
SICCD      116
TICKER     414
COMNAM     116
NAICS     2686
PRC        494
SHROUT     130
VOL        423
RET        196
dtype: int64


In [53]:
rows_with_missing_values = merged_df[merged_df.isnull().any(axis=1)]
print(rows_with_missing_values)

        PERMNO     date   SICCD TICKER       COMNAM  NAICS    PRC     SHROUT  \
0        10104  2004-01  7370.0   ORCL  ORACLE CORP    NaN  13.86  5227587.0   
1        10104  2004-02  7370.0   ORCL  ORACLE CORP    NaN  12.87  5195000.0   
2        10104  2004-03  7370.0   ORCL  ORACLE CORP    NaN  12.00  5194988.0   
3        10104  2004-04  7370.0   ORCL  ORACLE CORP    NaN  11.25  5194988.0   
4        10104  2004-05  7370.0   ORCL  ORACLE CORP    NaN  11.40  5171000.0   
...        ...      ...     ...    ...          ...    ...    ...        ...   
116720   93096  2009-10     NaN    NaN          NaN    NaN    NaN        NaN   
116891   93132  2009-10     NaN    NaN          NaN    NaN    NaN        NaN   
117062   93246  2010-01     NaN    NaN          NaN    NaN    NaN        NaN   
117230   93429  2010-05     NaN    NaN          NaN    NaN    NaN        NaN   
117394   93436  2010-05     NaN    NaN          NaN    NaN    NaN        NaN   

               VOL        RET  
0      

In [54]:
company_info = merged_df.groupby('TICKER')[['SICCD', 'COMNAM', 'NAICS']].first().reset_index()
merged_df = merged_df.drop(columns=['SICCD', 'COMNAM', 'NAICS'])
merged_df = pd.merge(merged_df, company_info, on='TICKER', how='left')

In [55]:
missing_values = merged_df.isnull().sum()
print(missing_values)

PERMNO      0
date        0
TICKER    414
PRC       494
SHROUT    130
VOL       423
RET       196
SICCD     414
COMNAM    414
NAICS     417
dtype: int64


In [56]:
company_info_by_permno = merged_df.groupby('PERMNO')[['TICKER', 'COMNAM']].first().reset_index()
merged_df = pd.merge(merged_df, company_info_by_permno, on='PERMNO', suffixes=('', '_from_permno'), how='left')
merged_df['TICKER'] = merged_df['TICKER'].fillna(merged_df['TICKER_from_permno'])
merged_df['COMNAM'] = merged_df['COMNAM'].fillna(merged_df['COMNAM_from_permno'])
merged_df = merged_df.drop(columns=['TICKER_from_permno', 'COMNAM_from_permno'])

In [57]:
missing_values_after_fill = merged_df.isnull().sum()
print(missing_values_after_fill)

PERMNO      0
date        0
TICKER      0
PRC       494
SHROUT    130
VOL       423
RET       196
SICCD     414
COMNAM      0
NAICS     417
dtype: int64


In [58]:
company_info_by_permno = merged_df.groupby('PERMNO')[['SICCD', 'NAICS']].first().reset_index()
merged_df = pd.merge(merged_df, company_info_by_permno, on='PERMNO', suffixes=('', '_from_permno'), how='left')

merged_df['SICCD'] = merged_df['SICCD'].fillna(merged_df['SICCD_from_permno'])
merged_df['NAICS'] = merged_df['NAICS'].fillna(merged_df['NAICS_from_permno'])

merged_df = merged_df.drop(columns=['SICCD_from_permno', 'NAICS_from_permno'])

In [59]:
missing_values_after_fill = merged_df.isnull().sum()
print(missing_values_after_fill)

PERMNO      0
date        0
TICKER      0
PRC       494
SHROUT    130
VOL       423
RET       196
SICCD       0
COMNAM      0
NAICS       0
dtype: int64


In [60]:
unique_tickers = merged_df['TICKER'].nunique()
print(f"不重复的 TICKER 数量: {unique_tickers}")

complete_data_per_ticker = merged_df.dropna(subset=['PRC', 'SHROUT', 'VOL', 'RET']).groupby('TICKER').size()

tickers_with_240_complete_records = complete_data_per_ticker[complete_data_per_ticker == 240].size
print(f"拥有完整 240 条数据的 TICKER 数量: {tickers_with_240_complete_records}")

不重复的 TICKER 数量: 629
拥有完整 240 条数据的 TICKER 数量: 328


In [61]:
num_cols = ['PRC', 'SHROUT', 'VOL', 'RET']
for col in num_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

In [62]:
def fill_missing_with_median(df, columns, groupby_col):
    for col in columns:
        df[col] = df.groupby(groupby_col)[col].transform(lambda x: x.fillna(x.median()))
    return df

columns_to_fill = ['PRC', 'SHROUT', 'VOL', 'RET']

merged_df = fill_missing_with_median(merged_df, columns_to_fill, 'TICKER')

In [63]:
missing_values_after_fill = merged_df.isnull().sum()
print(missing_values_after_fill)

PERMNO    0
date      0
TICKER    0
PRC       1
SHROUT    0
VOL       0
RET       1
SICCD     0
COMNAM    0
NAICS     0
dtype: int64


In [64]:
rows_with_missing_values = merged_df[merged_df.isnull().any(axis=1)]
print(rows_with_missing_values)

        PERMNO     date TICKER  PRC    SHROUT        VOL  RET   SICCD  \
108645   90441  2019-03  TFCFA  NaN  779853.0  2001929.0  NaN  2711.0   

                              COMNAM     NAICS  
108645  TWENTY FIRST CENTURY FOX INC  511110.0  


In [65]:
merged_df = merged_df.fillna(0)
missing_values_after_fill = merged_df.isnull().sum()
print(missing_values_after_fill)

PERMNO    0
date      0
TICKER    0
PRC       0
SHROUT    0
VOL       0
RET       0
SICCD     0
COMNAM    0
NAICS     0
dtype: int64


In [24]:
merged_df.to_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/2nd/Monthly_Merged1.csv', index=False)
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   PERMNO  117558 non-null  int64  
 1   date    117558 non-null  object 
 2   TICKER  117558 non-null  object 
 3   PRC     117558 non-null  float64
 4   SHROUT  117558 non-null  float64
 5   VOL     117558 non-null  float64
 6   RET     117558 non-null  float64
 7   SICCD   117558 non-null  float64
 8   COMNAM  117558 non-null  object 
 9   NAICS   117558 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 9.0+ MB
None


# External Data - PredictorData2023

In [66]:
predictor_df = pd.read_csv('Data/PredictorData2023 - Monthly.csv')
predictor_df = predictor_df[['yyyymm', 'ntis', 'tbl']]

In [67]:
predictor_df['yyyymm'] = pd.to_datetime(predictor_df['yyyymm'], format='%Y%m')

start_date = '2004-01-01'
end_date = '2023-12-31'
predictor_df = predictor_df[(predictor_df['yyyymm'] >= start_date) & (predictor_df['yyyymm'] <= end_date)]

In [68]:
predictor_df.rename(columns={'yyyymm': 'date'}, inplace=True)
predictor_df['date'] = predictor_df['date'].dt.to_period('M').astype(str)

In [28]:
predictor_df.to_csv('Data/Predictor_Monthly.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   PERMNO  117558 non-null  int64  
 1   date    117558 non-null  object 
 2   TICKER  117558 non-null  object 
 3   PRC     117558 non-null  float64
 4   SHROUT  117558 non-null  float64
 5   VOL     117558 non-null  float64
 6   RET     117558 non-null  float64
 7   SICCD   117558 non-null  float64
 8   COMNAM  117558 non-null  object 
 9   NAICS   117558 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 9.0+ MB
None


# 2nd Merge

In [69]:
merge2_df = pd.merge(merged_df, predictor_df, on='date', how='left')

merge2_df.to_csv('Data/Merge2_Monthly.csv', index=False)
print(merge2_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   PERMNO  117558 non-null  int64  
 1   date    117558 non-null  object 
 2   TICKER  117558 non-null  object 
 3   PRC     117558 non-null  float64
 4   SHROUT  117558 non-null  float64
 5   VOL     117558 non-null  float64
 6   RET     117558 non-null  float64
 7   SICCD   117558 non-null  float64
 8   COMNAM  117558 non-null  object 
 9   NAICS   117558 non-null  float64
 10  ntis    117558 non-null  float64
 11  tbl     117558 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 10.8+ MB
None


In [70]:
missing_values = merge2_df.isnull().sum()
print(missing_values)

PERMNO    0
date      0
TICKER    0
PRC       0
SHROUT    0
VOL       0
RET       0
SICCD     0
COMNAM    0
NAICS     0
ntis      0
tbl       0
dtype: int64


# INDPRO & T10Y3MM

In [71]:
indpro_df = pd.read_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/INDPRO.csv')
indpro_df.rename(columns={'DATE': 'date'}, inplace=True)
indpro_df['date'] = pd.to_datetime(indpro_df['date']).dt.to_period('M').astype(str)

In [72]:
t10y3mm_df = pd.read_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/T10Y3MM.csv')
t10y3mm_df.rename(columns={'DATE': 'date'}, inplace=True)
t10y3mm_df['date'] = pd.to_datetime(t10y3mm_df['date']).dt.to_period('M').astype(str)

In [73]:
merge3_df = pd.merge(merge2_df, indpro_df, on='date', how='left')

In [74]:
merge4_df = pd.merge(merge3_df, t10y3mm_df, on='date', how='left')

In [35]:
merge4_df.to_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/2nd/Merge4_with_INDPRO_T10Y3MM.csv', index=False)

In [75]:
print(merge4_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PERMNO   117558 non-null  int64  
 1   date     117558 non-null  object 
 2   TICKER   117558 non-null  object 
 3   PRC      117558 non-null  float64
 4   SHROUT   117558 non-null  float64
 5   VOL      117558 non-null  float64
 6   RET      117558 non-null  float64
 7   SICCD    117558 non-null  float64
 8   COMNAM   117558 non-null  object 
 9   NAICS    117558 non-null  float64
 10  ntis     117558 non-null  float64
 11  tbl      117558 non-null  float64
 12  INDPRO   117558 non-null  float64
 13  T10Y3MM  117558 non-null  float64
dtypes: float64(10), int64(1), object(3)
memory usage: 12.6+ MB
None


In [76]:
missing_values = merge4_df.isnull().sum()
print(missing_values)

PERMNO     0
date       0
TICKER     0
PRC        0
SHROUT     0
VOL        0
RET        0
SICCD      0
COMNAM     0
NAICS      0
ntis       0
tbl        0
INDPRO     0
T10Y3MM    0
dtype: int64


# Growth in Industrial Production (gpce) Calculation

In [77]:
if 'gpce' in merge4_df.columns:
    merge4_df.drop(columns=['gpce'], inplace=True)
if 'INDPRO' in merge4_df.columns:
    merge4_df.drop(columns=['INDPRO'], inplace=True)

In [78]:
indpro_df['gpce'] = (indpro_df['INDPRO'] - indpro_df['INDPRO'].shift(1)) / indpro_df['INDPRO'].shift(1)

merge5_df = pd.merge(merge4_df, indpro_df[['date', 'gpce']], on='date', how='left')

In [ ]:
merge5_df.to_csv('Data/Merge5_with_gpce.csv', index=False)

In [79]:
print(merge5_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PERMNO   117558 non-null  int64  
 1   date     117558 non-null  object 
 2   TICKER   117558 non-null  object 
 3   PRC      117558 non-null  float64
 4   SHROUT   117558 non-null  float64
 5   VOL      117558 non-null  float64
 6   RET      117558 non-null  float64
 7   SICCD    117558 non-null  float64
 8   COMNAM   117558 non-null  object 
 9   NAICS    117558 non-null  float64
 10  ntis     117558 non-null  float64
 11  tbl      117558 non-null  float64
 12  T10Y3MM  117558 non-null  float64
 13  gpce     117093 non-null  float64
dtypes: float64(10), int64(1), object(3)
memory usage: 12.6+ MB
None


In [80]:
missing_values = merge5_df.isnull().sum()
print(missing_values)

PERMNO       0
date         0
TICKER       0
PRC          0
SHROUT       0
VOL          0
RET          0
SICCD        0
COMNAM       0
NAICS        0
ntis         0
tbl          0
T10Y3MM      0
gpce       465
dtype: int64


In [81]:
merge5_df.fillna(0, inplace=True)

In [43]:
merge5_df.to_csv('Data/Merge5_with_gpce.csv', index=False)

In [82]:
missing_values = merge5_df.isnull().sum()
print(missing_values)

PERMNO     0
date       0
TICKER     0
PRC        0
SHROUT     0
VOL        0
RET        0
SICCD      0
COMNAM     0
NAICS      0
ntis       0
tbl        0
T10Y3MM    0
gpce       0
dtype: int64


# Balanced Panel???

In [55]:
company_counts = merge5_df['COMNAM'].value_counts()

unbalanced_companies = company_counts[company_counts < 240]

print("Companies with less than 240 records:")
print(unbalanced_companies)

Companies with less than 240 records:
COMNAM
N R G ENERGY INC                   239
SALESFORCE COM INC                 236
C B RICHARD ELLIS GROUP INC        236
MOLSON COORS BREWING CO            236
DOMINOS PIZZA INC                  235
                                  ... 
BANKNORTH GROUP INC ME              15
G E HEALTHCARE TECHNOLOGIES INC     13
KENVUE INC                           9
GLOBE LIFE INC                       4
VERALTO CORP                         4
Name: count, Length: 155, dtype: int64


In [56]:
print(company_counts)

COMNAM
MORGAN STANLEY DEAN WITTER & CO    480
LENNAR CORP                        480
MCCORMICK & CO INC                 480
BIO RAD LABORATORIES INC           480
WASTE MANAGEMENT INC DEL           480
                                  ... 
BANKNORTH GROUP INC ME              15
G E HEALTHCARE TECHNOLOGIES INC     13
KENVUE INC                           9
GLOBE LIFE INC                       4
VERALTO CORP                         4
Name: count, Length: 550, dtype: int64


In [57]:
unique_comnam_count = merge5_df['COMNAM'].nunique()

print(f"The number of unique company names (COMNAM) in the dataset is: {unique_comnam_count}")

The number of unique company names (COMNAM) in the dataset is: 550


In [58]:
company_counts = merge5_df['COMNAM'].value_counts()

complete_companies = company_counts[company_counts == 240]
print(f"The number of companies with exactly 240 records is: {complete_companies.shape[0]}")

print("Companies with exactly 240 records are:")
print(complete_companies.index.tolist())

The number of companies with exactly 240 records is: 382
Companies with exactly 240 records are:
['SYMANTEC CORP', 'TYLER TECHNOLOGIES INC', 'HOLOGIC INC', 'BORGWARNER INC', 'CABOT OIL & GAS CORP', 'CISCO SYSTEMS INC', 'PARAMETRIC TECHNOLOGY CORP', 'ELECTRONIC ARTS INC', 'EOG RESOURCES INC', 'VENTAS INC', 'TECHNE CORP', 'ADOBE SYSTEMS INC', 'IDEX CORP', 'TRIMBLE NAVIGATION LTD', 'SCHWAB CHARLES CORP NEW', 'CARNIVAL CORP', 'BAKER HUGHES INC', 'STRYKER CORP', 'STATE STREET CORP', 'B B & T CORP', 'EQUITY RESIDENTIAL', 'ECOLAB INC', 'CITIGROUP INC', 'RAYMOND JAMES FINANCIAL INC', 'VIACOM INC', 'AUTOZONE INC', 'ALLSTATE CORP', 'BOSTON SCIENTIFIC CORP', 'ROYAL CARIBBEAN CRUISES LTD', 'O REILLY AUTOMOTIVE INC', 'JABIL CIRCUIT INC', 'ACE LTD', 'MICROCHIP TECHNOLOGY INC', 'INTUIT INC', 'TYSON FOODS INC', 'STARBUCKS CORP', 'CAMDEN PROPERTY TRUST', 'D R HORTON INC', 'STERIS CORP', 'MOHAWK INDUSTRIES INC', 'REGENERON PHARMACEUTICALS INC', 'SYNOPSYS INC', 'ROPER INDUSTRIES INC NEW', 'GILEAD SCIENCE

# Momentum - Monthly

In [83]:
merge5_df = merge5_df.sort_values(by=['TICKER', 'date'])
merge5_df['12m_momentum'] = merge5_df.groupby('TICKER')['RET'].rolling(12).apply(lambda x: (x + 1).prod() - 1).reset_index(drop=True)

In [84]:
industry_groups = merge5_df.groupby(['NAICS', 'date'])
merge5_df['industry_momentum'] = industry_groups['12m_momentum'].transform('mean')

In [85]:
merge5_df['1m_reversal'] = merge5_df.groupby('TICKER')['RET'].shift(1)

In [86]:
missing_values_after_fill = merge5_df.isnull().sum()
print(missing_values_after_fill)

PERMNO                  0
date                    0
TICKER                  0
PRC                     0
SHROUT                  0
VOL                     0
RET                     0
SICCD                   0
COMNAM                  0
NAICS                   0
ntis                    0
tbl                     0
T10Y3MM                 0
gpce                    0
12m_momentum         6882
industry_momentum    1722
1m_reversal           629
dtype: int64


In [87]:
def fill_missing_with_median(df, columns, groupby_col):
    for col in columns:
        df[col] = df.groupby(groupby_col)[col].transform(lambda x: x.fillna(x.median()))
    return df

columns_to_fill = ['12m_momentum', 'industry_momentum', '1m_reversal']
merge5_df = fill_missing_with_median(merge5_df, columns_to_fill, 'TICKER')

In [88]:
missing_values_after_fill = merge5_df.isnull().sum()
print(missing_values_after_fill)

PERMNO               0
date                 0
TICKER               0
PRC                  0
SHROUT               0
VOL                  0
RET                  0
SICCD                0
COMNAM               0
NAICS                0
ntis                 0
tbl                  0
T10Y3MM              0
gpce                 0
12m_momentum         0
industry_momentum    0
1m_reversal          1
dtype: int64


In [90]:
merge5_df = merge5_df.fillna(0)

missing_values_after_fill = merge5_df.isnull().sum()
print(missing_values_after_fill)

PERMNO               0
date                 0
TICKER               0
PRC                  0
SHROUT               0
VOL                  0
RET                  0
SICCD                0
COMNAM               0
NAICS                0
ntis                 0
tbl                  0
T10Y3MM              0
gpce                 0
12m_momentum         0
industry_momentum    0
1m_reversal          0
dtype: int64


# Market Value & Excess Return(target variable)

In [91]:
merge5_df['EXRET'] = merge5_df['RET'] - merge5_df['tbl']

merge5_df['MV'] = merge5_df['PRC'] * merge5_df['SHROUT']

In [92]:
merge5_df.to_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/2nd/5test_with_EXRET_MV.csv', index=False)

print(merge5_df.head())

       PERMNO     date TICKER    PRC    SHROUT       VOL       RET   SICCD  \
96096   87432  2004-01      A  36.86  480041.0  936159.0  0.260602  3825.0   
96097   87432  2004-02      A  34.19  480041.0  624089.0 -0.072436  3825.0   
96098   87432  2004-03      A  31.63  480754.0  761789.0 -0.074876  3825.0   
96099   87432  2004-04      A  27.01  480754.0  502169.0 -0.146064  3825.0   
96100   87432  2004-05      A  25.70  481000.0  788345.0 -0.048501  3825.0   

                         COMNAM     NAICS      ntis     tbl  T10Y3MM  \
96096  AGILENT TECHNOLOGIES INC  334515.0  0.006887  0.0088     3.25   
96097  AGILENT TECHNOLOGIES INC  334515.0  0.009361  0.0093     3.14   
96098  AGILENT TECHNOLOGIES INC  334515.0  0.012282  0.0094     2.88   
96099  AGILENT TECHNOLOGIES INC  334515.0  0.013766  0.0094     3.39   
96100  AGILENT TECHNOLOGIES INC  334515.0  0.016330  0.0102     3.68   

           gpce  12m_momentum  industry_momentum  1m_reversal     EXRET  \
96096  0.000000     -0.

# Liquidity - Daily

In [93]:
daily_df = pd.read_csv('Data/Daily - Liquidity.csv')
daily_df['date'] = pd.to_datetime(daily_df['date'])

/var/folders/43/g17v034j3x7dj44_fyl7pl_m0000gn/T/ipykernel_35173/3256828450.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  daily_df = pd.read_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/Daily - Liquidity.csv')


In [94]:
daily_df['DV'] = daily_df['VOL'] * daily_df['PRC']
daily_df['BAS'] = daily_df['ASK'] - daily_df['BID']
daily_df['year_month'] = daily_df['date'].dt.to_period('M')

monthly_dv = daily_df.groupby(['TICKER', 'year_month'])['DV'].sum().reset_index()
monthly_bas = daily_df.groupby(['TICKER', 'year_month'])['BAS'].mean().reset_index()
monthly_df = pd.merge(monthly_dv, monthly_bas, on=['TICKER', 'year_month'])

In [95]:
monthly_df.to_csv('Data/Monthly - Liquidity1.csv', index=False)
print(monthly_df.head())

  TICKER year_month            DV       BAS
0      A    2004-01  3.178481e+09  0.024000
1      A    2004-02  2.232170e+09  0.038421
2      A    2004-03  2.389114e+09  0.034783
3      A    2004-04  1.513678e+09  0.042857
4      A    2004-05  2.018090e+09  0.026000


In [96]:
missing_values = monthly_df.isnull().sum()
print(missing_values)

TICKER         0
year_month     0
DV             0
BAS           13
dtype: int64


In [98]:
median_bas = monthly_df.groupby('TICKER')['BAS'].transform('median')
monthly_df['BAS'].fillna(median_bas, inplace=True)

In [99]:
missing_values_after = monthly_df.isnull().sum()
print(missing_values_after)

TICKER        0
year_month    0
DV            0
BAS           0
dtype: int64


# Total Return Volatility - Standard Deviation

In [100]:
daily_df = daily_df[pd.to_numeric(daily_df['RET'], errors='coerce').notnull()]
daily_df['RET'] = daily_df['RET'].astype(float)

In [101]:
monthly_volatility = daily_df.groupby(['TICKER', 'year_month'])['RET'].std().reset_index()
monthly_volatility.rename(columns={'RET': 'TRV'}, inplace=True)


In [102]:
monthly_volatility['year'] = monthly_volatility['year_month'].dt.year
annual_volatility = monthly_volatility.groupby(['TICKER', 'year'])['TRV'].mean().reset_index()
annual_volatility.rename(columns={'TRV': 'annual_TRV'}, inplace=True)

In [103]:
missing_values_after = annual_volatility.isnull().sum()
print(missing_values_after)

TICKER        0
year          0
annual_TRV    4
dtype: int64


In [104]:
annual_volatility['annual_TRV'].fillna(0, inplace=True)

In [105]:
missing_values_after = annual_volatility.isnull().sum()
print(missing_values_after)

TICKER        0
year          0
annual_TRV    0
dtype: int64


In [106]:
annual_volatility = annual_volatility[['TICKER', 'year', 'annual_TRV']]

annual_volatility.to_csv('Data/annual_volatility.csv', index=False)
print(annual_volatility.head())

  TICKER  year  annual_TRV
0      A  2004    0.023131
1      A  2005    0.015626
2      A  2006    0.017079
3      A  2007    0.015205
4      A  2008    0.026656


# Regression - Idiosyncratic Volatility & Market Beta

In [107]:
daily_df2 = pd.read_csv('Data/Daily - Liquidity.csv')

daily_df2['date'] = pd.to_datetime(daily_df2['date'])
daily_df2['year'] = daily_df2['date'].dt.year
daily_df2['year_month'] = daily_df2['date'].dt.to_period('M')

daily_df2['RET'] = pd.to_numeric(daily_df2['RET'], errors='coerce')
daily_df2['sprtrn'] = pd.to_numeric(daily_df2['sprtrn'], errors='coerce')

/var/folders/43/g17v034j3x7dj44_fyl7pl_m0000gn/T/ipykernel_35173/2042031660.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  daily_df2 = pd.read_csv('/Users/yuyangdou/RSM/BAM/11Model/Data/USA/Daily - Liquidity.csv')


In [108]:
daily_df2['IV'] = np.nan
daily_df2['MB'] = np.nan

companies = daily_df2['TICKER'].unique()
for company in companies:
    company_data = daily_df2[daily_df2['TICKER'] == company]
    
    
    company_data = company_data.dropna(subset=['RET', 'sprtrn'])
    
    if len(company_data) > 1: 
        X = sm.add_constant(company_data['sprtrn'])
        y = company_data['RET']
        
        model = sm.OLS(y, X).fit()
        
        daily_df2.loc[daily_df2['TICKER'] == company, 'MB'] = model.params['sprtrn']
        daily_df2.loc[daily_df2['TICKER'] == company, 'IV'] = model.resid.std()

annual_iv_mb = daily_df2.groupby(['TICKER', 'year']).agg({
    'IV': 'mean',
    'MB': 'mean'
}).reset_index()

In [109]:
missing_values_after = annual_iv_mb.isnull().sum()
print(missing_values_after)

TICKER    0
year      0
IV        1
MB        1
dtype: int64


In [110]:
median_iv_mb = annual_iv_mb.groupby('TICKER')[['IV', 'MB']].median().reset_index()

def fill_missing_values(row, median_df):
    if pd.isnull(row['IV']):
        row['IV'] = median_df.loc[median_df['TICKER'] == row['TICKER'], 'IV'].values[0]
    if pd.isnull(row['MB']):
        row['MB'] = median_df.loc[median_df['TICKER'] == row['TICKER'], 'MB'].values[0]
    return row

annual_iv_mb = annual_iv_mb.apply(fill_missing_values, axis=1, median_df=median_iv_mb)

In [111]:
missing_values_after = annual_iv_mb.isnull().sum()
print(missing_values_after)

TICKER    0
year      0
IV        1
MB        1
dtype: int64


In [113]:
annual_iv_mb['IV'].fillna(0, inplace=True)

In [114]:
annual_iv_mb['MB'].fillna(0, inplace=True)

In [115]:
missing_values_after = annual_iv_mb.isnull().sum()
print(missing_values_after)

TICKER    0
year      0
IV        0
MB        0
dtype: int64


In [116]:
annual_iv_mb.to_csv('Data/annual_iv_mb.csv', index=False)

# Final Merge - excluding Financial Ratio

In [117]:
final_df = pd.read_csv('Data/5test_with_EXRET_MV.csv')

In [118]:
monthly_liquidity = pd.read_csv('Data/Monthly - Liquidity1.csv')
monthly_liquidity.rename(columns={'year_month': 'date'}, inplace=True)

annual_volatility = pd.read_csv('Data/annual_volatility.csv')

annual_iv_mb = pd.read_csv('Data/annual_iv_mb.csv')

In [119]:
final_df['date'] = pd.to_datetime(final_df['date']).dt.to_period('M').astype(str)
monthly_liquidity['date'] = pd.to_datetime(monthly_liquidity['date']).dt.to_period('M').astype(str)
annual_volatility['year'] = annual_volatility['year'].astype(str)
annual_iv_mb['year'] = annual_iv_mb['year'].astype(str)


In [120]:
final_df['year'] = final_df['date'].str[:4]

In [121]:
final_df = pd.merge(final_df, monthly_liquidity[['TICKER', 'date', 'DV', 'BAS']], on=['TICKER', 'date'], how='left')
final_df = pd.merge(final_df, annual_volatility[['TICKER', 'year', 'annual_TRV']], on=['TICKER', 'year'], how='left')
final_df = pd.merge(final_df, annual_iv_mb[['TICKER', 'year', 'IV', 'MB']], on=['TICKER', 'year'], how='left')

In [122]:
print(final_df.info())
print(final_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117558 entries, 0 to 117557
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   PERMNO             117558 non-null  int64  
 1   date               117558 non-null  object 
 2   TICKER             117558 non-null  object 
 3   PRC                117558 non-null  float64
 4   SHROUT             117558 non-null  float64
 5   VOL                117558 non-null  float64
 6   RET                117558 non-null  float64
 7   SICCD              117558 non-null  float64
 8   COMNAM             117558 non-null  object 
 9   NAICS              117558 non-null  float64
 10  ntis               117558 non-null  float64
 11  tbl                117558 non-null  float64
 12  T10Y3MM            117558 non-null  float64
 13  gpce               117558 non-null  float64
 14  12m_momentum       117558 non-null  float64
 15  industry_momentum  117558 non-null  float64
 16  1m

In [123]:
missing_values_after = final_df.isnull().sum()
print(missing_values_after)

PERMNO                 0
date                   0
TICKER                 0
PRC                    0
SHROUT                 0
VOL                    0
RET                    0
SICCD                  0
COMNAM                 0
NAICS                  0
ntis                   0
tbl                    0
T10Y3MM                0
gpce                   0
12m_momentum           0
industry_momentum      0
1m_reversal            0
EXRET                  0
MV                     0
year                   0
DV                   226
BAS                  239
annual_TRV            55
IV                    54
MB                    54
dtype: int64


In [124]:
final_df['year'] = final_df['year'].astype(int)

In [125]:
def fill_missing_with_median(df, columns, groupby_col):
    for col in columns:
        df[col] = df.groupby(groupby_col)[col].transform(lambda x: x.fillna(x.median()))
    return df

columns_to_fill = ['DV', 'BAS', 'annual_TRV', 'IV', 'MB']

final_df = fill_missing_with_median(final_df, columns_to_fill, 'TICKER')

In [126]:
missing_values_after = final_df.isnull().sum()
print(missing_values_after)

PERMNO               0
date                 0
TICKER               0
PRC                  0
SHROUT               0
VOL                  0
RET                  0
SICCD                0
COMNAM               0
NAICS                0
ntis                 0
tbl                  0
T10Y3MM              0
gpce                 0
12m_momentum         0
industry_momentum    0
1m_reversal          0
EXRET                0
MV                   0
year                 0
DV                   0
BAS                  0
annual_TRV           0
IV                   0
MB                   0
dtype: int64


In [127]:
final_df.to_csv('Data/final_merged_dataset1.csv', index=False)

# Final Final Merge - including Financial Ratio

In [128]:
financial_ratio_df = pd.read_csv('Data/Financial Ratio - 3Annual.csv')

In [129]:
financial_ratio_df.rename(columns={
    'Data Year - Fiscal': 'year',
    'Ticker Symbol': 'TICKER',
    'Company Name': 'COMNAM'
}, inplace=True)

In [130]:
final_df = pd.merge(final_df, financial_ratio_df[['year', 'TICKER', 'COMNAM', 'Book-to-Market', 'Earnings-Price', 'Sales-to-Price']], on=['year', 'TICKER', 'COMNAM'], how='left')

In [131]:
missing_values_after_merge = final_df.isnull().sum()
print(missing_values_after_merge)

PERMNO                   0
date                     0
TICKER                   0
PRC                      0
SHROUT                   0
VOL                      0
RET                      0
SICCD                    0
COMNAM                   0
NAICS                    0
ntis                     0
tbl                      0
T10Y3MM                  0
gpce                     0
12m_momentum             0
industry_momentum        0
1m_reversal              0
EXRET                    0
MV                       0
year                     0
DV                       0
BAS                      0
annual_TRV               0
IV                       0
MB                       0
Book-to-Market       62007
Earnings-Price       62007
Sales-to-Price       62007
dtype: int64


In [132]:
final_df['year'] = final_df['year'].astype(int)

In [143]:
def fill_missing_with_median(df, columns, groupby_col):
    for col in columns:
        df[col] = df.groupby(groupby_col)[col].transform(lambda x: x.fillna(x.median()))
    return df

columns_to_fill = ['Book-to-Market', 'Earnings-Price', 'Sales-to-Price']

final_df = fill_missing_with_median(final_df, columns_to_fill, 'TICKER')

In [144]:
missing_values_after_merge = final_df.isnull().sum()
print(missing_values_after_merge)

PERMNO                   0
date                     0
TICKER                   0
PRC                      0
SHROUT                   0
VOL                      0
RET                      0
SICCD                    0
COMNAM                   0
NAICS                    0
ntis                     0
tbl                      0
T10Y3MM                  0
gpce                     0
12m_momentum             0
industry_momentum        0
1m_reversal              0
EXRET                    0
MV                       0
year                     0
DV                       0
BAS                      0
annual_TRV               0
IV                       0
MB                       0
Book-to-Market       13718
Earnings-Price       13718
Sales-to-Price       13718
dtype: int64


In [145]:
final_df.to_csv('Data/FINAL_DF.csv', index=False)

# Balanced Panel Filter: 500 -> 325 (no missing value)

In [146]:
company_counts = final_df['TICKER'].value_counts()

complete_companies = company_counts[company_counts == 240]

print(f"The number of companies with exactly 240 records is: {complete_companies.shape[0]}")

print("Companies with exactly 240 records are:")
print(complete_companies.index.tolist())

The number of companies with exactly 240 records is: 328
Companies with exactly 240 records are:
['MAS', 'LRCX', 'MAR', 'MAA', 'MCD', 'MCHP', 'MCK', 'LUV', 'A', 'LOW', 'LNT', 'LMT', 'LLY', 'LH', 'LEE', 'KR', 'KO', 'KMX', 'KMB', 'MCO', 'MHK', 'MDT', 'MTD', 'NTRS', 'NTAP', 'NSC', 'NOC', 'NKE', 'NI', 'NFLX', 'NEM', 'NDSN', 'MU', 'MTB', 'MET', 'MSFT', 'MRO', 'MRK', 'MOH', 'MO', 'MMM', 'MMC', 'MLM', 'KIM', 'MGM', 'KLAC', 'JNPR', 'KEY', 'HPQ', 'HOLX', 'HIG', 'HD', 'HBAN', 'HAS', 'HAL', 'GWW', 'GS', 'GRMN', 'GPN', 'GPC', 'GLW', 'GIS', 'GILD', 'GE', 'GD', 'FRT', 'FMC', 'FITB', 'FFIV', 'FE', 'FDX', 'FDS', 'HON', 'HRL', 'K', 'HSIC', 'NVDA', 'JNJ', 'JKHY', 'JBL', 'JBHT', 'ITW', 'ITT', 'ITI', 'ISRG', 'IRM', 'IR', 'IPG', 'IP', 'INTU', 'INTC', 'INCY', 'ILMN', 'IFF', 'IEX', 'IDXX', 'IBM', 'HUM', 'HSY', 'NUE', 'OKE', 'NVR', 'STLD', 'UHS', 'UDR', 'TYL', 'TXT', 'TXN', 'TTWO', 'TSN', 'TSCO', 'TROW', 'TRMB', 'TMO', 'TJX', 'TGT', 'TFX', 'TER', 'TECH', 'TDY', 'T', 'SYY', 'SYK', 'SWKS', 'SWK', 'STX', 'UNH', 

In [147]:
company_counts = final_df['TICKER'].value_counts()

complete_companies = company_counts[company_counts == 240].index

final_df_filtered = final_df[final_df['TICKER'].isin(complete_companies)]

print(final_df_filtered.info())

<class 'pandas.core.frame.DataFrame'>
Index: 78720 entries, 0 to 117288
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PERMNO             78720 non-null  int64  
 1   date               78720 non-null  object 
 2   TICKER             78720 non-null  object 
 3   PRC                78720 non-null  float64
 4   SHROUT             78720 non-null  float64
 5   VOL                78720 non-null  float64
 6   RET                78720 non-null  float64
 7   SICCD              78720 non-null  float64
 8   COMNAM             78720 non-null  object 
 9   NAICS              78720 non-null  float64
 10  ntis               78720 non-null  float64
 11  tbl                78720 non-null  float64
 12  T10Y3MM            78720 non-null  float64
 13  gpce               78720 non-null  float64
 14  12m_momentum       78720 non-null  float64
 15  industry_momentum  78720 non-null  float64
 16  1m_reversal        78720 n

In [148]:
missing_values_after_merge = final_df_filtered.isnull().sum()
print(missing_values_after_merge)

PERMNO                  0
date                    0
TICKER                  0
PRC                     0
SHROUT                  0
VOL                     0
RET                     0
SICCD                   0
COMNAM                  0
NAICS                   0
ntis                    0
tbl                     0
T10Y3MM                 0
gpce                    0
12m_momentum            0
industry_momentum       0
1m_reversal             0
EXRET                   0
MV                      0
year                    0
DV                      0
BAS                     0
annual_TRV              0
IV                      0
MB                      0
Book-to-Market       8400
Earnings-Price       8400
Sales-to-Price       8400
dtype: int64


In [149]:
companies_without_missing_values = final_df_filtered.dropna(subset=final_df_filtered.columns)

complete_company_counts = companies_without_missing_values['TICKER'].value_counts()

complete_companies_no_missing = complete_company_counts[complete_company_counts == 240]

print(f"The number of companies with exactly 240 records and no missing values is: {complete_companies_no_missing.shape[0]}")

The number of companies with exactly 240 records and no missing values is: 293


In [150]:
final_df_no_missing = final_df_filtered[final_df_filtered['TICKER'].isin(complete_companies_no_missing.index)]

In [151]:
missing_values_after_merge = final_df_no_missing.isnull().sum()
print(missing_values_after_merge)

PERMNO               0
date                 0
TICKER               0
PRC                  0
SHROUT               0
VOL                  0
RET                  0
SICCD                0
COMNAM               0
NAICS                0
ntis                 0
tbl                  0
T10Y3MM              0
gpce                 0
12m_momentum         0
industry_momentum    0
1m_reversal          0
EXRET                0
MV                   0
year                 0
DV                   0
BAS                  0
annual_TRV           0
IV                   0
MB                   0
Book-to-Market       0
Earnings-Price       0
Sales-to-Price       0
dtype: int64


In [152]:
final_df_no_missing.to_csv('Data/final_df_no_missing_293.csv', index=False)